In [1]:
import pandas as pd
import numpy as np

# Dataset size

In [2]:
clean_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=' + str(0) +\
        '&$select=count(tree_id)').replace(' ', '%20')
df_size = pd.read_json(clean_url)
df_size

,count_tree_id
0,683788


## Shape of retrieved data
There are over 600,000 rows in the entire data set. I am only going to retrieve the relevant columns and relevant aggregation through the API call.

Columns retrieved:

borocode: 1 (Manhattan), 2 (Bronx), 3 (Brooklyn), 4 (Queens), 5 (Staten Island)
spc_common: specie name (132 unique species)
health: good, fair, poor
steward: Indicates the number of unique signs of stewardship observed for this tree (none, 1or2, 3or4, 4ormore)
Through trial and error, I determined that there is total of 4565 rows when data is grouped by borocode, spc_common, health, and steward. The limit value is set to 1000 since this is the maximum number of rows that can be retrieved through each API call. The offset value increments by 1000. max_row is set to 5000. This allows us to retrieve all 4565 rows.

NOTE: I had to use 'borocode' because I was getting error when I used the 'borough' column to group the data by.

In [4]:
offset = 1000
max_row = 5000

In [9]:
for x in range(0, max_row, offset):
    #print('x is ' + str(x))
    soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=1000&$offset=' + str(x) +\
        '&$select=borocode,spc_common,health,steward,count(tree_id)' +\
        '&$group=borocode,spc_common,health,steward').replace(' ', '%20')
    soql_trees = pd.read_json(soql_url)
    if(x==0):
        df = pd.DataFrame(columns=list(soql_trees.columns.values))
    df = df.append(soql_trees)

In [10]:
df = df.reset_index(drop=True)

Size of the dataset

In [11]:
len(df)

4565

In [12]:
# unique names
len(list(df.spc_common.unique())) 

133

In [13]:
# drop na to avoid errors in the visualization 
df = df.dropna(axis = 0, how = 'any')

In [14]:
df.head(5)

,borocode,spc_common,health,steward,count_tree_id
0,3,London planetree,Good,3or4,644
1,3,Kentucky coffeetree,Good,4orMore,16
2,3,sycamore maple,Poor,None,41
3,5,Chinese tree lilac,Poor,1or2,1
4,5,red maple,Good,None,5094


## Data preparation
Build a dash app for a arborist studying the health of various tree species (as defined by the variable ‘spc_common’) across each borough (defined by the variable ‘borough’). This arborist would like to answer the following two questions for each species and in each borough:

What proportion of trees are in good, fair, or poor health according to the ‘health’ variable?
Are stewards (steward activity measured by the ‘steward’ variable) having an impact on the health of trees?

## Work process

For every specice and in each borough, what proportion of trees are in good, fair, or poor health?

The application will allow arborist to select one specie, and the application will display proportion of trees that are in good, fair, or poor health across all boroughs. Arborist will be able to compare health of particular specie across all five boroughs.

Bar graphs will be used to present the proportions. The orientation of the bar graphs will be vertical. The bar graphs will be first grouped by boroughs for each health status.

The goal of the code below is to create a dataframe that has the columns: borocode, spc_common, health, ratio.

Ratio is the proportion of spc_common in the given borough that has the given heath level. For example, a ratio for red maple in Queens with a health of good is the proportion of good red maple trees in Queens.

In [15]:
df_totals = df.groupby(['borocode', 'spc_common'])['count_tree_id'].sum()
df_total_by_borocode_specie_health = df.groupby(['borocode', 'spc_common', 'health'])['count_tree_id'].sum()

In [16]:
df_totals.head(5)

borocode  spc_common            
1         'Schubert' chokecherry     163
          American beech              22
          American elm              1698
          American hophornbeam        84
          American hornbeam           85
Name: count_tree_id, dtype: int64

In [17]:
df_total_by_borocode_specie_health.head(5)


borocode  spc_common              health
1         'Schubert' chokecherry  Fair       40
                                  Good      111
                                  Poor       12
          American beech          Fair        4
                                  Good       15
Name: count_tree_id, dtype: int64

In [18]:
df_totals = df_totals.reset_index(drop=False)
df_total_by_borocode_specie_health = df_total_by_borocode_specie_health.reset_index(drop=False)

In [19]:
df_totals.head(5)


,borocode,spc_common,count_tree_id
0,1,'Schubert' chokecherry,163
1,1,American beech,22
2,1,American elm,1698
3,1,American hophornbeam,84
4,1,American hornbeam,85


In [20]:
df_total_by_borocode_specie_health.head(5)


,borocode,spc_common,health,count_tree_id
0,1,'Schubert' chokecherry,Fair,40
1,1,'Schubert' chokecherry,Good,111
2,1,'Schubert' chokecherry,Poor,12
3,1,American beech,Fair,4
4,1,American beech,Good,15


In [21]:
df_totals.columns = ['borocode', 'spc_common', 'total_for_specie_in_borough']
df_total_by_borocode_specie_health.columns = ['borocode', 'spc_common', 'health', 'total']

In [22]:
tree_proportions = pd.merge(df_total_by_borocode_specie_health, df_totals, on=['borocode', 'spc_common'])


In [23]:
tree_proportions.head(5)

,borocode,spc_common,health,total,total_for_specie_in_borough
0,1,'Schubert' chokecherry,Fair,40,163
1,1,'Schubert' chokecherry,Good,111,163
2,1,'Schubert' chokecherry,Poor,12,163
3,1,American beech,Fair,4,22
4,1,American beech,Good,15,22


In [24]:
tree_proportions['ratio'] = tree_proportions['total']/ tree_proportions['total_for_specie_in_borough']

In [25]:
tree_proportions.head(10)


,borocode,spc_common,health,total,total_for_specie_in_borough,ratio
0,1,'Schubert' chokecherry,Fair,40,163,0.245399
1,1,'Schubert' chokecherry,Good,111,163,0.680982
2,1,'Schubert' chokecherry,Poor,12,163,0.073620
3,1,American beech,Fair,4,22,0.181818
4,1,American beech,Good,15,22,0.681818
5,1,American beech,Poor,3,22,0.136364
6,1,American elm,Fair,259,1698,0.152532
7,1,American elm,Good,1361,1698,0.801531
8,1,American elm,Poor,78,1698,0.045936
9,1,American hophornbeam,Fair,12,84,0.142857


## Question 2
I would like to use a scatter plot to represent the overall health status of the selected specie across all the boroughs. An overall health index is determined by assigning a numeric value to each health level (Poor=1, Fair=2, Good=3) and then calculating a weighted average for the selected specie for each borough. The overall health index score has a minimum score of 1 and a maximum score of 3.

In [26]:
list(df['steward'].unique())


['3or4', '4orMore', 'None', '1or2']

In [27]:
df.head(10)


,borocode,spc_common,health,steward,count_tree_id
0,3,London planetree,Good,3or4,644
1,3,Kentucky coffeetree,Good,4orMore,16
2,3,sycamore maple,Poor,None,41
3,5,Chinese tree lilac,Poor,1or2,1
4,5,red maple,Good,None,5094
5,4,Atlantic white cedar,Good,3or4,10
6,5,boxelder,Fair,None,6
7,2,purple-leaf plum,Fair,3or4,3
8,2,'Schubert' chokecherry,Poor,None,25
9,2,hawthorn,Fair,1or2,16


In [28]:
df.sort_values(by=['borocode', 'spc_common', 'steward']).head(10)


,borocode,spc_common,health,steward,count_tree_id
620,1,'Schubert' chokecherry,Good,1or2,56
2510,1,'Schubert' chokecherry,Poor,1or2,11
3561,1,'Schubert' chokecherry,Fair,1or2,23
1851,1,'Schubert' chokecherry,Good,3or4,21
2353,1,'Schubert' chokecherry,Fair,3or4,2
433,1,'Schubert' chokecherry,Good,4orMore,3
1422,1,'Schubert' chokecherry,Good,None,31
3342,1,'Schubert' chokecherry,Fair,None,15
4085,1,'Schubert' chokecherry,Poor,None,1
912,1,American beech,Good,1or2,7


In [29]:
df_total_by_steward = df.groupby(['borocode', 'spc_common', 'steward'])['count_tree_id'].sum()
df_total_by_steward = df_total_by_steward.reset_index(drop=False)
df_total_by_steward.columns = ['borocode', 'spc_common', 'steward', 'steward_total']
df_total_by_steward.head(10)

,borocode,spc_common,steward,steward_total
0,1,'Schubert' chokecherry,1or2,90
1,1,'Schubert' chokecherry,3or4,23
2,1,'Schubert' chokecherry,4orMore,3
3,1,'Schubert' chokecherry,None,47
4,1,American beech,1or2,10
5,1,American beech,None,12
6,1,American elm,1or2,495
7,1,American elm,3or4,102
8,1,American elm,4orMore,15
9,1,American elm,None,1086


In [30]:
df_steward = pd.merge(df, df_total_by_steward, on=['borocode', 'spc_common', 'steward'])
df_steward.head(10)

,borocode,spc_common,health,steward,count_tree_id,steward_total
0,3,London planetree,Good,3or4,644,772
1,3,London planetree,Poor,3or4,18,772
2,3,London planetree,Fair,3or4,110,772
3,3,Kentucky coffeetree,Good,4orMore,16,18
4,3,Kentucky coffeetree,Fair,4orMore,2,18
5,3,sycamore maple,Poor,None,41,418
6,3,sycamore maple,Fair,None,75,418
7,3,sycamore maple,Good,None,302,418
8,5,Chinese tree lilac,Poor,1or2,1,32
9,5,Chinese tree lilac,Fair,1or2,7,32


In [31]:
di = {'Poor':1, 'Fair':2, 'Good':3}
df_steward['health_level'] = df_steward['health'].map(di)
df_steward.sort_values(by=['borocode', 'spc_common', 'steward']).head(10)
df_steward['health_index'] = (df_steward['count_tree_id']/df_steward['steward_total']) * df_steward['health_level']
df_steward.sort_values(by=['borocode', 'spc_common', 'steward']).head(10)
df_overall_health_index = df_steward.groupby(['borocode', 'spc_common', 'steward'])['health_index'].sum()
df_overall_health_index = df_overall_health_index.reset_index(drop=False)
df_overall_health_index.columns = ['borocode', 'spc_common', 'steward', 'overall_health_index']
di2 = {'3or4':3, '4orMore':4, 'None':1, '1or2':2}
df_overall_health_index['steward_level'] = df_overall_health_index['steward'].map(di2)
di3 = { 1:'Manhattan', 2:'Bronx', 3:'Brooklyn', 4:'Queens', 5:'Staten Island'}
df_overall_health_index['borough'] = df_overall_health_index['borocode'].map(di3)
df_overall_health_index['spc_common'] = df_overall_health_index['spc_common'].apply(lambda x: x.title())
df_overall_health_index.head(10)

,borocode,spc_common,steward,overall_health_index,steward_level,borough
0,1,'Schubert' Chokecherry,1or2,2.500000,2,Manhattan
1,1,'Schubert' Chokecherry,3or4,2.913043,3,Manhattan
2,1,'Schubert' Chokecherry,4orMore,3.000000,4,Manhattan
3,1,'Schubert' Chokecherry,None,2.638298,1,Manhattan
4,1,American Beech,1or2,2.700000,2,Manhattan
5,1,American Beech,None,2.416667,1,Manhattan
6,1,American Elm,1or2,2.707071,2,Manhattan
7,1,American Elm,3or4,2.686275,3,Manhattan
8,1,American Elm,4orMore,2.800000,4,Manhattan
9,1,American Elm,None,2.783610,1,Manhattan
